In [0]:
%run "/WMU/OCR"

In [0]:
from azure.storage.blob import BlobServiceClient
from azure.storage.blob import generate_blob_sas, BlobSasPermissions
from datetime import datetime, timedelta
import json
import pandas as pd
from openpyxl import load_workbook
from azure.storage.blob import BlobServiceClient
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
from openpyxl import load_workbook
from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

from azure.storage.blob import BlobServiceClient
from azure.ai.formrecognizer import (
    DocumentModelAdministrationClient,
    ModelBuildMode,
)
from pyspark.sql.types import StructType, StructField, StringType

import pandas as pd
import re
from io import BytesIO


All the post processing of data

In [0]:
def format_dates(value,year):

    if(value is not None):
        trimmed_value =value.replace(" ",'')
        if(len(trimmed_value)!=10):
            trimmed_value = trimmed_value[0:6]+year
            return trimmed_value
    return value

In [0]:
def get_text_from_img(url):
    try:
        text=analyze_read(url)
        return text
    except Exception :
        print("I am sorry, not able to process your request")


In [0]:

def analyze_read(url):
    endpoint = "https://wmu.cognitiveservices.azure.com/"
    key = "18f28d4cfa6b4d0e8fa07419c212d0ef"

    formUrl = url

    document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )

    poller = document_analysis_client.begin_analyze_document_from_url(
            "prebuilt-read", formUrl)
    result = poller.result()

    result_text= result.content
    data=[]
    for page in result.pages:
        for line_idx, line in enumerate(page.lines):

            for word in page.words:

                data.append({'content':word.content,'confidence':word.confidence,'line_number': line_idx + 1})
    df = spark.createDataFrame(data)
    spark.conf.set("fs.azure.account.key.wmu.dfs.core.windows.net","0qlFK6Nrn/fQTh7bMne+D9xcuYue5tvKGbQ8LMQuh/ZJGYoLiEnurDTJnE00HZSPHiV2UfBttziI+AStIybg3Q==")
    df.write.mode('overwrite').csv("abfss://wmu-fm@wmu.dfs.core.windows.net/output")
    return result_text




'UNITED STATES SECURITIES AND EXCHANGE COMMISSION Washington, D.C. 20549\nFORM 10-Q\n☒ QUARTERLY REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the Quarterly Period Ended March 31, 2020\nOR\n☐ TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the Transition Period From\nto\nCommission File Number 001-37845\nMICROSOFT CORPORATION\nWASHINGTON (STATE OF INCORPORATION)\n91-1144442\n(I.R.S. ID)\nONE MICROSOFT WAY, REDMOND, WASHINGTON 98052-6399 (425) 882-8080 www.microsoft.com/investor\nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of each class\nTrading Symbol\nName of exchange on which registered\nMSFT\nNASDAQ\nMSFT\nNASDAQ\nMSFT\nNASDAQ\nMSFT\nNASDAQ\nCommon stock, $0.00000625 par value per share 2.125% Notes due 2021 3.125% Notes due 2028 2.625% Notes due 2033\nSecurities registered pursuant to Section 12(g) of the Act:\nNONE\nIndicate by check mark whether the registrant (1) has filed a

In [0]:

from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient
def get_labelled_data(imgUrl):
    endpoint = "https://wmu.cognitiveservices.azure.com/"
    key = "18f28d4cfa6b4d0e8fa07419c212d0ef"

    formUrl = imgUrl

    document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )
    try:
    
        poller = document_analysis_client.begin_analyze_document_from_url("prebuilt-invoice", formUrl)
        invoices = poller.result()

        for idx, invoice in enumerate(invoices.documents):
            print("--------Recognizing invoice #{}--------".format(idx + 1))
            vendor_name = invoice.fields.get("VendorName")
            if vendor_name:
                print(
                    "Vendor Name: {} has confidence: {}".format(
                        vendor_name.value, vendor_name.confidence
                    )
                )
            vendor_address = invoice.fields.get("VendorAddress")
            if vendor_address:
                print(
                    "Vendor Address: {} has confidence: {}".format(
                        vendor_address.value, vendor_address.confidence
                    )
                )
            vendor_address_recipient = invoice.fields.get("VendorAddressRecipient")
            if vendor_address_recipient:
                print(
                    "Vendor Address Recipient: {} has confidence: {}".format(
                        vendor_address_recipient.value, vendor_address_recipient.confidence
                    )
                )
            customer_name = invoice.fields.get("CustomerName")
            if customer_name:
                print(
                    "Customer Name: {} has confidence: {}".format(
                        customer_name.value, customer_name.confidence
                    )
                )
            customer_id = invoice.fields.get("CustomerId")
            if customer_id:
                print(
                    "Customer Id: {} has confidence: {}".format(
                        customer_id.value, customer_id.confidence
                    )
                )
            customer_address = invoice.fields.get("CustomerAddress")
            if customer_address:
                print(
                    "Customer Address: {} has confidence: {}".format(
                        customer_address.value, customer_address.confidence
                    )
                )
            customer_address_recipient = invoice.fields.get("CustomerAddressRecipient")
            if customer_address_recipient:
                print(
                    "Customer Address Recipient: {} has confidence: {}".format(
                        customer_address_recipient.value,
                        customer_address_recipient.confidence,
                    )
                )
            invoice_id = invoice.fields.get("InvoiceId")
            if invoice_id:
                print(
                    "Invoice Id: {} has confidence: {}".format(
                        invoice_id.value, invoice_id.confidence
                    )
                )
            invoice_date = invoice.fields.get("InvoiceDate")
            if invoice_date:
                print(
                    "Invoice Date: {} has confidence: {}".format(
                        invoice_date.value, invoice_date.confidence
                    )
                )
            invoice_total = invoice.fields.get("InvoiceTotal")
            if invoice_total:
                print(
                    "Invoice Total: {} has confidence: {}".format(
                        invoice_total.value, invoice_total.confidence
                    )
                )
            due_date = invoice.fields.get("DueDate")
            if due_date:
                print(
                    "Due Date: {} has confidence: {}".format(
                        due_date.value, due_date.confidence
                    )
                )
            purchase_order = invoice.fields.get("PurchaseOrder")
            if purchase_order:
                print(
                    "Purchase Order: {} has confidence: {}".format(
                        purchase_order.value, purchase_order.confidence
                    )
                )
            billing_address = invoice.fields.get("BillingAddress")
            if billing_address:
                print(
                    "Billing Address: {} has confidence: {}".format(
                        billing_address.value, billing_address.confidence
                    )
                )
            billing_address_recipient = invoice.fields.get("BillingAddressRecipient")
            if billing_address_recipient:
                print(
                    "Billing Address Recipient: {} has confidence: {}".format(
                        billing_address_recipient.value,
                        billing_address_recipient.confidence,
                    )
                )
            shipping_address = invoice.fields.get("ShippingAddress")
            if shipping_address:
                print(
                    "Shipping Address: {} has confidence: {}".format(
                        shipping_address.value, shipping_address.confidence
                    )
                )
            shipping_address_recipient = invoice.fields.get("ShippingAddressRecipient")
            if shipping_address_recipient:
                print(
                    "Shipping Address Recipient: {} has confidence: {}".format(
                        shipping_address_recipient.value,
                        shipping_address_recipient.confidence,
                    )
                )
            print("Invoice items:")
            for idx, item in enumerate(invoice.fields.get("Items").value):
                print("...Item #{}".format(idx + 1))
                item_description = item.value.get("Description")
                if item_description:
                    print(
                        "......Description: {} has confidence: {}".format(
                            item_description.value, item_description.confidence
                        )
                    )
                item_quantity = item.value.get("Quantity")
                if item_quantity:
                    print(
                        "......Quantity: {} has confidence: {}".format(
                            item_quantity.value, item_quantity.confidence
                        )
                    )
                unit = item.value.get("Unit")
                if unit:
                    print(
                        "......Unit: {} has confidence: {}".format(
                            unit.value, unit.confidence
                        )
                    )
                unit_price = item.value.get("UnitPrice")
                if unit_price:
                    print(
                        "......Unit Price: {} has confidence: {}".format(
                            unit_price.value, unit_price.confidence
                        )
                    )
                product_code = item.value.get("ProductCode")
                if product_code:
                    print(
                        "......Product Code: {} has confidence: {}".format(
                            product_code.value, product_code.confidence
                        )
                    )
                item_date = item.value.get("Date")
                if item_date:
                    print(
                        "......Date: {} has confidence: {}".format(
                            item_date.value, item_date.confidence
                        )
                    )
                tax = item.value.get("Tax")
                if tax:
                    print(
                        "......Tax: {} has confidence: {}".format(tax.value, tax.confidence)
                    )
                amount = item.value.get("Amount")
                if amount:
                    print(
                        "......Amount: {} has confidence: {}".format(
                            amount.value, amount.confidence
                        )
                    )
            subtotal = invoice.fields.get("SubTotal")
            if subtotal:
                print(
                    "Subtotal: {} has confidence: {}".format(
                        subtotal.value, subtotal.confidence
                    )
                )
            total_tax = invoice.fields.get("TotalTax")
            if total_tax:
                print(
                    "Total Tax: {} has confidence: {}".format(
                        total_tax.value, total_tax.confidence
                    )
                )
            previous_unpaid_balance = invoice.fields.get("PreviousUnpaidBalance")
            if previous_unpaid_balance:
                print(
                    "Previous Unpaid Balance: {} has confidence: {}".format(
                        previous_unpaid_balance.value, previous_unpaid_balance.confidence
                    )
                )
            amount_due = invoice.fields.get("AmountDue")
            if amount_due:
                print(
                    "Amount Due: {} has confidence: {}".format(
                        amount_due.value, amount_due.confidence
                    )
                )
            service_start_date = invoice.fields.get("ServiceStartDate")
            if service_start_date:
                print(
                    "Service Start Date: {} has confidence: {}".format(
                        service_start_date.value, service_start_date.confidence
                    )
                )
            service_end_date = invoice.fields.get("ServiceEndDate")
            if service_end_date:
                print(
                    "Service End Date: {} has confidence: {}".format(
                        service_end_date.value, service_end_date.confidence
                    )
                )
            service_address = invoice.fields.get("ServiceAddress")
            if service_address:
                print(
                    "Service Address: {} has confidence: {}".format(
                        service_address.value, service_address.confidence
                    )
                )
            service_address_recipient = invoice.fields.get("ServiceAddressRecipient")
            if service_address_recipient:
                print(
                    "Service Address Recipient: {} has confidence: {}".format(
                        service_address_recipient.value,
                        service_address_recipient.confidence,
                    )
                )
            remittance_address = invoice.fields.get("RemittanceAddress")
            if remittance_address:
                print(
                    "Remittance Address: {} has confidence: {}".format(
                        remittance_address.value, remittance_address.confidence
                    )
                )
            remittance_address_recipient = invoice.fields.get("RemittanceAddressRecipient")
            if remittance_address_recipient:
                print(
                    "Remittance Address Recipient: {} has confidence: {}".format(
                        remittance_address_recipient.value,
                        remittance_address_recipient.confidence,
                    )
                )
            print("----------------------------------------")
    except Exception: 
        return 'Sorry, try again later. '
